In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/news_tagging_model
# !mkdir news_tagging_model

In [ ]:
!pip install transformers
!pip install sentencepiece

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [ ]:
def compute_metrics(outs):
    predictions, labels = outs
    predictions = np.argmax(predictions, axis = -1)

    ## computes overall scores (accuracy, f1, recall, precision)
    accuracy = accuracy_score(labels, predictions) * 100
    f1 = f1_score(labels, predictions, average = "macro") * 100
    recall = recall_score(labels, predictions, average = "macro") * 100
    precision = precision_score(labels, predictions, average = "macro") * 100

    return {
        "accuracy" : float(accuracy),
        "f1" : float(f1),
        "recall" : float(recall),
        "precision" : float(precision),
    }

In [ ]:
def encode_labels(labels):
  labels_set = set(labels)
  endcoded_labels = labels

  for j in range(len(endcoded_labels)):
    if endcoded_labels[j] == 'TRUE':
      endcoded_labels[j] = 1
    else:
      endcoded_labels[j] = 0

  return endcoded_labels

# encode_labels(["x","health","y","x","z"])

In [ ]:
def load_data(path):
    """
    read CSV file and return the tweets and labels lists
    """
    df = pd.read_csv(path)
    titles = df['title'].tolist()
    labels = encode_labels(df['Politics_Label'].tolist())
    print("max(labels)")

    print(max(labels))
    return titles, labels

In [ ]:
class MultiDialectDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels != None:
          item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
df = pd.read_csv("Brazilian politics - estadao-export-2022-08-09.csv")
df

In [ ]:
df.Politics_Label.value_counts()

In [ ]:
df =  df[df.Politics_Label.isin(['FALSE','TRUE'])]
df

In [ ]:
val_tweets = df.title.tolist()
val_labels = []
for i in  df.Politics_Label.tolist():
  if i == 'FALSE':
    val_labels.append(0)
  elif i == 'TRUE':
    val_labels.append(1)
# val_labels.replace('FALSE',0)

In [ ]:
print("Loading Model...")
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/news_tagging_model/finetune_out/best_ckpt", num_labels = 2)
print("Loading Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/news_tagging_model/finetune_out/best_ckpt")


In [ ]:
print("Tokenizeing the inputs...")
val_encodings = tokenizer(val_tweets,
                          truncation=True,
                          padding=True,
                            #max_length=model.config.max_position_embeddings
                          )

In [ ]:
val_ds = MultiDialectDataset(val_encodings, val_labels)


In [ ]:
trainer = Trainer(model=model)
trainer.model = model.cuda()


In [ ]:
val_pred = trainer.predict(val_ds)


In [ ]:
pred_0 = []
pred_1 = []
ispolitics_pred = []
for pred in val_pred.predictions:
  pred_0.append(pred[0])
  pred_1.append(pred[1])
  if pred[0]>pred[1]:
    ispolitics_pred.append(0)
  else:
    ispolitics_pred.append(1)

In [ ]:
df_results = pd.DataFrame({'title':val_tweets,'ispolitics_truth':val_labels,'ispolitics_pred':ispolitics_pred,'pred_0':pred_0,'pred_1':pred_1})

In [ ]:
df_results.to_csv('Brazilian politics - estadao-export-2022-08-09 - prediction_output.csv')

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(val_labels, ispolitics_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()



In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(val_labels, ispolitics_pred)

